In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

from tqdm import tqdm
import os, sys

In [2]:
data = pd.read_csv('data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [4]:
# Создадим датафрейм с покупками юзеров на тестовом датасете (последние 3 недели)
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

# перевод numpy массива с списки - сериализация
result['actual'] = result['actual'].apply(lambda x: list(x))

result.head(10)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8..."
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884..."
7,14,"[840601, 867293, 933067, 951590, 952408, 96569..."
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13..."
9,16,"[1062973, 1082185, 13007710]"


In [5]:
# Предскажем случайные рекомендации
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""

    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [6]:
%%time

items = data_train.item_id.unique()
print(items)

# создаём столбец рекомендаций: для каждой строчки user_id создаём случайные рекомендации
result['rand rec'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result.head(5)

[ 1004906  1033142  1036325 ... 15722756 17170636 15716393]
Wall time: 3.69 s


,user_id,actual,rand rec
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[2452321, 1071713, 9881905, 1110603, 66392]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[8205620, 908649, 9487467, 78017, 1148035]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[9836714, 10120669, 851226, 1025816, 925866]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[10254149, 423783, 12949308, 13157784, 5651399]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[13040169, 145505, 2017057, 1052855, 9878513]"


In [7]:
# Проверяем сколько рекомендованных товаров входят в список актуальных товаров
hit_rate = 0
for i in range(result.shape[0]):
    flags = np.isin(result.loc[i,'actual'], result.loc[i,'rand rec'])
    hit_rate += (flags.sum() > 0).astype(int)
hit_rate

10

In [8]:
def weighted_random_recommendation(items, p, n=5):
    """Случайные рекоммендации
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    recs = np.random.choice(items, size=n, replace=False, p=p)
    
    return recs.tolist()

In [9]:
# функция для генерации items и массива вероятностей
def data_prep(df, func):
    # создаём список уникальных товаров
    items = df.item_id.unique()
    # print('items 1 = ', items, items.size)

    # переводим в np массив
    items = np.array(items)

    # создаём датафрейм с количеством каждого товара
    count_df = df.groupby('item_id').item_id.agg(['count'])

    # добавляем в датафрейм строчку с пересчитанными товарами в веса
    count_df['weight'] = func(count_df['count'] / count_df['count'].sum())
    # print(count_df.size)

    # перводим items в датафрейм (сохраняем порядок item'ов)
    items_df = pd.DataFrame(items, columns = ['item_id'])
    # print('items_df = ', items_df)

    # создаём датафрейм item - вероятность с сохранением порядка item'ов
    probability_df = pd.merge(items_df, count_df, on='item_id', how='left')

    # заполняем пустые item's нулями
    probability_df.fillna(value=0, inplace=True)
    # print('probability_df = ', probability_df, probability_df.size)

    # берём столбец вероятностей и переводим его в numpy-array
    p = np.array(probability_df.to_numpy()[:, 2])

    # нормализация вероятностей для того чтобы они  были = 1
    p /= p.sum()  # normalize
    
    return items, p

In [10]:
%%time

# ВЕСА - ЛОГАРИФМ от количества покупок
items, p = data_prep(data_train, np.log)

# создаём столбец рекомендаций: для каждой строчки user_id создаём случайные рекомендации
result['weighted log rand rec'] = \
        result['user_id'].apply(lambda x: weighted_random_recommendation(items, p, n=5))

result.head(3)

Wall time: 1.78 s


,user_id,actual,rand rec,weighted log rand rec
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[2452321, 1071713, 9881905, 1110603, 66392]","[5577021, 6463800, 12486291, 12385595, 12731689]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[8205620, 908649, 9487467, 78017, 1148035]","[7140623, 3766950, 6034741, 1079406, 2559282]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[9836714, 10120669, 851226, 1025816, 925866]","[894133, 5589150, 15972567, 13007237, 14112257]"


In [11]:
%%time

# your_code

# ВЕСА - КВАДРАТ от количества покупок
items, p = data_prep(data_train, np.square)

# создаём столбец рекомендаций: для каждой строчки user_id создаём случайные рекомендации
result['weighted square rand rec'] = \
            result['user_id'].apply(lambda x: weighted_random_recommendation(items, p, n=5))

result.head(2)

Wall time: 2.1 s


,user_id,actual,rand rec,weighted log rand rec,weighted square rand rec
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[2452321, 1071713, 9881905, 1110603, 66392]","[5577021, 6463800, 12486291, 12385595, 12731689]","[1003441, 1082185, 1029743, 6534178, 981760]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[8205620, 908649, 9487467, 78017, 1148035]","[7140623, 3766950, 6034741, 1079406, 2559282]","[948670, 6534178, 995242, 1082185, 878996]"


## Задание 2. Улучшение бейзлайнов и ItemItem

##### - Попробуйте улучшить бейзлайны, считая случаный на топ-5000 товаров

In [12]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [13]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [14]:
%%time

items = top_5000

# создаём столбец рекомендаций: для каждой строчки user_id создаём случайные рекомендации
result['rand rec top 5000'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result.head(2)

Wall time: 840 ms


,user_id,actual,rand rec,weighted log rand rec,weighted square rand rec,rand rec top 5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[2452321, 1071713, 9881905, 1110603, 66392]","[5577021, 6463800, 12486291, 12385595, 12731689]","[1003441, 1082185, 1029743, 6534178, 981760]","[991575, 1122790, 1131602, 836281, 945821]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[8205620, 908649, 9487467, 78017, 1148035]","[7140623, 3766950, 6034741, 1079406, 2559282]","[948670, 6534178, 995242, 1082185, 878996]","[819765, 12188583, 9194207, 880423, 879922]"


In [15]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = top_5000[:5]

result['popular rec top 5000'] = result['user_id'].apply(lambda x: popular_recs)
result.head(3)

Wall time: 999 µs


,user_id,actual,rand rec,weighted log rand rec,weighted square rand rec,rand rec top 5000,popular rec top 5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[2452321, 1071713, 9881905, 1110603, 66392]","[5577021, 6463800, 12486291, 12385595, 12731689]","[1003441, 1082185, 1029743, 6534178, 981760]","[991575, 1122790, 1131602, 836281, 945821]","[6534178, 6533889, 6534166, 6544236, 1404121]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[8205620, 908649, 9487467, 78017, 1148035]","[7140623, 3766950, 6034741, 1079406, 2559282]","[948670, 6534178, 995242, 1082185, 878996]","[819765, 12188583, 9194207, 880423, 879922]","[6534178, 6533889, 6534166, 6544236, 1404121]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[9836714, 10120669, 851226, 1025816, 925866]","[894133, 5589150, 15972567, 13007237, 14112257]","[1053690, 1029743, 1082185, 6534178, 904360]","[947797, 1013572, 12810422, 1013531, 913278]","[6534178, 6533889, 6534166, 6544236, 1404121]"


In [17]:
# функция для генерации items и массива вероятностей
def data_prep_top_5000(df, func):
    # создаём список уникальных товаров
    items = df.item_id.unique()
    # print('items 1 = ', items, items.size)

    # переводим в np массив
    items = np.array(items)[:5000]           # < == фильтруем 5000

    # создаём датафрейм с количеством каждого товара
    count_df = df.groupby('item_id').item_id.agg(['count'])

    # добавляем в датафрейм строчку с пересчитанными товарами в веса
    count_df['weight'] = func(count_df['count'] / count_df['count'].sum())
    # print(count_df.size)

    # перводим items в датафрейм (сохраняем порядок item'ов)
    items_df = pd.DataFrame(items, columns = ['item_id'])
    # print('items_df = ', items_df)

    # создаём датафрейм item - вероятность с сохранением порядка item'ов
    probability_df = pd.merge(items_df, count_df, on='item_id', how='left').head(5000) # <= фильтруем 5000

    # заполняем пустые item's нулями
    probability_df.fillna(value=0, inplace=True)
    # print('probability_df = ', probability_df, probability_df.size)

    # берём столбец вероятностей и переводим его в numpy-array
    p = np.array(probability_df.to_numpy()[:, 2])

    # нормализация вероятностей для того чтобы они  были = 1
    p /= p.sum()  # normalize
    
    return items, p

In [18]:
%%time

# ВЕСА - ЛОГАРИФМ от количества покупок
items, p = data_prep_top_5000(data_train, np.log)

# создаём столбец рекомендаций: для каждой строчки user_id создаём случайные рекомендации
result['weighted log rand rec top 5000'] = \
        result['user_id'].apply(lambda x: weighted_random_recommendation(items, p, n=5))

result.head(3)

Wall time: 429 ms


,user_id,actual,rand rec,weighted log rand rec,weighted square rand rec,rand rec top 5000,popular rec top 5000,weighted log rand rec top 5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[2452321, 1071713, 9881905, 1110603, 66392]","[5577021, 6463800, 12486291, 12385595, 12731689]","[1003441, 1082185, 1029743, 6534178, 981760]","[991575, 1122790, 1131602, 836281, 945821]","[6534178, 6533889, 6534166, 6544236, 1404121]","[1008689, 9487278, 6442662, 5571798, 6548453]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[8205620, 908649, 9487467, 78017, 1148035]","[7140623, 3766950, 6034741, 1079406, 2559282]","[948670, 6534178, 995242, 1082185, 878996]","[819765, 12188583, 9194207, 880423, 879922]","[6534178, 6533889, 6534166, 6544236, 1404121]","[1133852, 824546, 933060, 940796, 920850]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[9836714, 10120669, 851226, 1025816, 925866]","[894133, 5589150, 15972567, 13007237, 14112257]","[1053690, 1029743, 1082185, 6534178, 904360]","[947797, 1013572, 12810422, 1013531, 913278]","[6534178, 6533889, 6534166, 6544236, 1404121]","[1037840, 916133, 1009567, 882108, 1075786]"


In [19]:
%%time

# ВЕСА - КВАДРАТНЫЙ КОРЕНЬ от количества покупок
items, p = data_prep_top_5000(data_train, np.sqrt)

# создаём столбец рекомендаций: для каждой строчки user_id создаём случайные рекомендации
result['weighted sqrt rand rec top 5000'] = \
            result['user_id'].apply(lambda x: weighted_random_recommendation(items, p, n=5))

result.head(2)

Wall time: 431 ms


,user_id,actual,rand rec,weighted log rand rec,weighted square rand rec,rand rec top 5000,popular rec top 5000,weighted log rand rec top 5000,weighted sqrt rand rec top 5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[2452321, 1071713, 9881905, 1110603, 66392]","[5577021, 6463800, 12486291, 12385595, 12731689]","[1003441, 1082185, 1029743, 6534178, 981760]","[991575, 1122790, 1131602, 836281, 945821]","[6534178, 6533889, 6534166, 6544236, 1404121]","[1008689, 9487278, 6442662, 5571798, 6548453]","[1041651, 873447, 1031740, 943393, 991268]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[8205620, 908649, 9487467, 78017, 1148035]","[7140623, 3766950, 6034741, 1079406, 2559282]","[948670, 6534178, 995242, 1082185, 878996]","[819765, 12188583, 9194207, 880423, 879922]","[6534178, 6533889, 6534166, 6544236, 1404121]","[1133852, 824546, 933060, 940796, 920850]","[996070, 1100975, 5591611, 1058997, 1030242]"


In [20]:
%%time

# your_code

# ВЕСА - КВАДРАТ от количества покупок
items, p = data_prep_top_5000(data_train, np.square)

# создаём столбец рекомендаций: для каждой строчки user_id создаём случайные рекомендации
result['weighted square rand rec top 5000'] = \
            result['user_id'].apply(lambda x: weighted_random_recommendation(items, p, n=5))

result.head(2)

Wall time: 478 ms


,user_id,actual,rand rec,weighted log rand rec,weighted square rand rec,rand rec top 5000,popular rec top 5000,weighted log rand rec top 5000,weighted sqrt rand rec top 5000,weighted square rand rec top 5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[2452321, 1071713, 9881905, 1110603, 66392]","[5577021, 6463800, 12486291, 12385595, 12731689]","[1003441, 1082185, 1029743, 6534178, 981760]","[991575, 1122790, 1131602, 836281, 945821]","[6534178, 6533889, 6534166, 6544236, 1404121]","[1008689, 9487278, 6442662, 5571798, 6548453]","[1041651, 873447, 1031740, 943393, 991268]","[1029743, 9527290, 961554, 1082185, 1006184]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[8205620, 908649, 9487467, 78017, 1148035]","[7140623, 3766950, 6034741, 1079406, 2559282]","[948670, 6534178, 995242, 1082185, 878996]","[819765, 12188583, 9194207, 880423, 879922]","[6534178, 6533889, 6534166, 6544236, 1404121]","[1133852, 824546, 933060, 940796, 920850]","[996070, 1100975, 5591611, 1058997, 1030242]","[877447, 6534178, 995785, 1082185, 1029743]"


In [34]:
# Функции из 1-ого вебинара
import os, sys

In [36]:
# Такой вывод, на мой взгляд, удобнее
for column in result.columns[2:]:
    print(column, round(result.apply(lambda x: precision_at_k(x[column], x['actual'],  5), axis=1).mean(), 5))

rand rec 0.00098
weighted log rand rec 0.00049
weighted square rand rec 0.1335
rand rec top 5000 0.00597
popular rec top 5000 0.04613
weighted log rand rec top 5000 0.00304
weighted sqrt rand rec top 5000 0.01391
weighted square rand rec top 5000 0.14006


In [37]:
# Выведем данные как на вебинаре
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.001:rand rec
0.0005:weighted log rand rec
0.1335:weighted square rand rec
0.006:rand rec top 5000
0.0461:popular rec top 5000
0.003:weighted log rand rec top 5000
0.0139:weighted sqrt rand rec top 5000
0.1401:weighted square rand rec top 5000


При ограничении 5000 товаров часть метрик улучшились.

##### - Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [39]:
# перенумеруем пользователей и товары, чтобы они задавались числами от 0 до номера количества товаров.
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [56]:
%%time

for K in tqdm(range(1, 11)):
    model = ItemItemRecommender(K=K, num_threads=8) # K - кол-во билжайших соседей и количество потоков (ядер процессора)

    ## item-item recommender ожидает, что мы подадим в столбцах - user, в строках item. Поэтому транспонируем
    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=False)

    recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, # не рекомендуем то, что он уже покупал
                        filter_items=None, 
                        recalculate_user=True)
    
    result[f'itemitem K={K}'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.43it/s]

Wall time: 6.99 s


In [57]:
for column in result.columns[2:]:
    print(column, round(result.apply(lambda x: precision_at_k(x[column], x['actual'],  5), axis=1).mean(), 5))

itemitem 0.13692
itemitem K=1 0.17999
itemitem K=2 0.19201
itemitem K=3 0.18609
itemitem K=4 0.14496
itemitem K=5 0.13692
itemitem K=6 0.14202
itemitem K=7 0.14486
itemitem K=8 0.14721
itemitem K=9 0.14848
itemitem K=10 0.15093


Лучшая метрика при K=2 0.19201